In [19]:
import functools
import json
import os
from pathlib import Path

from gossiplearning.config import Config
from utils.gossip_training import get_node_dataset, round_trip_fn, model_transmission_fn, \
    run_simulation
from utils.model_creators import create_LSTM
from utils.multiprocessing import run_in_parallel

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [20]:
with open("config.json", "r") as f:
    config = json.load(f)

In [21]:
config = Config.model_validate(config)

workspace_path = Path(config.workspace_dir)
workspace_path.mkdir(exist_ok=True, parents=True)
with (workspace_path / "config.json").open("w") as f:
    json.dump(config.model_dump(), f, indent=True)

In [22]:
model_creator = functools.partial(create_LSTM, config=config)

In [23]:
model_creator().summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 4, 9)]            0         
                                                                 
 sequential_2 (Sequential)   (None, 50)                32200     
                                                                 
 fn_0 (Dense)                (None, 1)                 51        
                                                                 
Total params: 32,251
Trainable params: 32,251
Non-trainable params: 0
_________________________________________________________________


In [24]:
datasets_folder = Path(f"data/datasets/1func_10nodes_3k")
networks_folder = Path(f"data/networks/10_nodes_3k")

In [25]:
# for i in range(1):
#     run_simulation(
#         config=config,
#         simulation_number=i,
#         network_folder=networks_folder / str(i),
#         round_trip_fn=round_trip_fn,
#         model_transmission_fn=model_transmission_fn,
#         node_data_fn=functools.partial(
#             get_node_dataset, base_folder=datasets_folder, simulation_number=i, input_steps=4),
#         model_creator=model_creator,
#     )

In [26]:
run_in_parallel(
    [
        functools.partial(
           run_simulation,
           config=config,
           simulation_number=i,
           network_folder=networks_folder / str(i),
           round_trip_fn=round_trip_fn,
           model_transmission_fn=model_transmission_fn,
           node_data_fn=functools.partial(
               get_node_dataset, 
               base_folder=datasets_folder, 
               simulation_number=i, 
               input_steps=4
           ),
           model_creator=model_creator)
        for i in range(2)
    ]
)

Metal device set to: Apple M1 Pro
Metal device set to: Apple M1 Pro
1/10 stopped! (Time 101s)
2/10 stopped! (Time 114s)
1/10 stopped! (Time 106s)
3/10 stopped! (Time 119s)
2/10 stopped! (Time 108s)
3/10 stopped! (Time 108s)
4/10 stopped! (Time 111s)
4/10 stopped! (Time 122s)
5/10 stopped! (Time 125s)
6/10 stopped! (Time 127s)
5/10 stopped! (Time 121s)
7/10 stopped! (Time 132s)
6/10 stopped! (Time 124s)
8/10 stopped! (Time 158s)
7/10 stopped! (Time 143s)
9/10 stopped! (Time 161s)
8/10 stopped! (Time 148s)
9/10 stopped! (Time 149s)
10/10 stopped! (Time 184s)
10/10 stopped! (Time 159s)

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB
